In [6]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
import googletrans
import time

# Cleaning run: removing of unwanted data

In [7]:
# Loading of the data in a DataFrame object
data = pd.read_csv('../original_classification_IARC_english.csv')

# Replace every missing data for group by the string 'unknown'
#data['Group'][[x for x in range(data.shape[0]) if data.Group.isnull()[x]]] = 'unknown'
#data['Group'][data['Group'].isnull()] = 'unknown'
data = data.fillna('unknown')

# Replace not separating commas by dots
for i in range(data.shape[0]):
    for j in range(len(data.loc[i, 'Agent'])):
        if data.loc[i, 'Agent'][j] == ",":
            tmp = list(data.loc[i, 'Agent'])
            tmp[j] = '.'
            data.loc[i, 'Agent'] = "".join(tmp)

# Removing unused substances by keyword
forbidden_keywords = ['Tobacco', 'tobacco', 'Sunlamps', 'Radio', 'Human', 'Printing', 'Textile', 'Ultraviolet', 'Gamma',\
                      'Wood', 'Solar', 'Welding', 'implants', 'Outdoor', 'radiation', 'wool',\
                      'infection', 'Petroleum', 'Leather', 'production', 'Paint', 'fuel', 'Lead', 'frequency',\
                      'virus', 'manufacture', 'fields', 'temperature', 'Nickel', 'Implanted', 'implant', 'industries', 'Glass',\
                      'engine', 'mining', 'founding', 'Shiftwork', 'Fuel', 'lighting', 'Fission', 'Firefighter',\
                      'Gasoline', 'Hair', 'Engine', 'therapy', 'Dental', 'emissions', 'exposures', 'Coal',\
                      'joinery', 'Soot', 'decay', 'Very hot beverages', 'Orange', 'contraceptives', 'Oestrogen', 'Mate']

# identifying every row that contains one of the forbidden keywords and then drop that rows
rows_to_remove = []
for i in range(data.shape[0]):
    for j in forbidden_keywords:
        if j in data['Agent'][i]:
            rows_to_remove += [i]
data = data.drop(rows_to_remove)


# Reindexing
data.index = range(data.shape[0])
data.index.name='id'


# Removing the italic stones <i>a</i>
agent_serie = data['Agent']

for index, element in enumerate(agent_serie):
    string_relay1 = element.replace('<i>', '')
    string_relay2 = string_relay1.replace('(see also', '(see')
    agent_serie[index] = string_relay2.replace('</i>', '')

            
data

,Agent,Group
id,,
0,Formaldehyde,1
1,Phenobarbital,2B
2,Mitomycin C,2B
3,Cyclophosphamide,1
4,DDT (4.4'-Dichlorodiphenyltrichloroethane),2A
5,Benzo[a]pyrene,1
6,Phenylbutazone,3
7,Clomiphene citrate,3
8,6-Mercaptopurine,3


# Filling unknown groups

In [8]:
# Slicing data to retrieve components that have an unknown group
sliced_data = data.loc[[x for x in data.index if (('(see ' in data['Agent'][x] and data['Group'][x] == 'unknown') \
                                            or ('(see also ' in data['Agent'][x] and data['Group'][x] == 'unknown'))]]
                                                

for i in range(sliced_data.shape[0]):
    # Get '(see ' position
    sub_first_index = sliced_data.iloc[i]['Agent'].find('(see ')
    sub_last_index = len(sliced_data.iloc[i]['Agent'])-1
    associated_compound = sliced_data.iloc[i]['Agent'][sub_first_index+5:sub_last_index]
    print(associated_compound)
    
    # Get index of associated_compound
    k = 0
    test = False
    while (k < data.shape[0] and test == False):
        #print(k)
        if (data.iloc[k]['Agent'].startswith(associated_compound)):
            print(k)
            sliced_data.iloc[i]['Group'] = data.iloc[k]['Group']
            sliced_data.iloc[i]['Agent'] = sliced_data.iloc[i]['Agent'][:sub_first_index-1]
            print(sliced_data.iloc[i]['Agent'])
            test = True
        k = k + 1
        if (k == data.shape[0]):
            print(associated_compound, 'not found')

data.loc[[x for x in data.index if (('(see ' in data['Agent'][x] and data['Group'][x] == 'unknown') \
                                            or ('(see also ' in data['Agent'][x] and data['Group'][x] == 'unknown'))]]\
                = sliced_data
            
data.to_csv('../clean_english_IARC_classification.csv')

Busulfan
29
1.4-Butanediol dimethanesulfonate
Busulfan
29
Myleran
Dieldrin. and aldrin metabolized to dieldrin
59
Dieldrin
Paracetamol
236
Acetaminophen
Bis(chloromethyl)ether; chloromethyl methyl ether
461
Chloromethyl methyl ether
Di(2-ethylhexyl) phthalate
Di(2-ethylhexyl) phthalate not found
Methylarsonic acid
312
Monomethylarsonic acid
Dieldrin. and aldrin metabolized to dieldrin
59
Aldrin
Chlornaphazine
442
N.N-Bis(2-chloroethyl)-2-naphthylamine
Sulfur mustard
447
Mustard gas
Arsenic and inorganic arsenic compounds
627
Gallium arsenide
Monographs on 1.3-Butadiene
Monographs on 1.3-Butadiene not found
Benzidine. dyes metabolized to
846
CI Direct Black 38
Benzidine. dyes metabolized to
846
CI Direct Blue 6
Acid mists
838
Strong-inorganic-acid mists containing sulfuric acid
Cobalt metal with tungsten carbide
633
Tungsten carbide with cobalt metal
Palygorskite
686
Attapulgite
Semustine
698
1-(2-Chloroethyl)-3-(4-methylcyclohexyl)-1-nitrosourea (Methyl-CCNU)
Asbestos
532
Talc containi

In [9]:
data

,Agent,Group
id,,
0,Formaldehyde,1
1,Phenobarbital,2B
2,Mitomycin C,2B
3,Cyclophosphamide,1
4,DDT (4.4'-Dichlorodiphenyltrichloroethane),2A
5,Benzo[a]pyrene,1
6,Phenylbutazone,3
7,Clomiphene citrate,3
8,6-Mercaptopurine,3


# Translation

### First half

In [15]:
agent_serie = data['Agent']
french_data = data.copy()
translator = googletrans.Translator()

print('FIRST HALF')
for index in range(int(data.shape[0] / 2)):
    print()
    print(index)
    print(data['Agent'].iloc[index])
    try:
        translated_elem = translator.translate(data['Agent'].iloc[index], dest='fr', src='en')
        print('\b', '->', translated_elem.text)
        french_data['Agent'].iloc[index] = translated_elem.text
    except:
        time.sleep(10)
        print('Warning: not translated')
        translated_elem = translator.translate(element, dest='fr', src='en')
        print('\b', '->', translated_elem.text)
        french_data['Agent'].iloc[index] = translated_elem.text
    






FIRST HALF

0
Formaldehyde
 -> Formaldéhyde

1
Phenobarbital
 -> Phénobarbital

2
Mitomycin C
 -> Mitomycine C

3
Cyclophosphamide
 -> Cyclophosphamide

4
DDT (4.4'-Dichlorodiphenyltrichloroethane)
 -> DDT (4,4'-dichlorodiphényltrichloroéthane)

5
Benzo[a]pyrene
 -> Benzo [a] pyrène

6
Phenylbutazone
 -> Phénylbutazone

7
Clomiphene citrate
 -> Citrate de clomifène

8
6-Mercaptopurine
 -> 6-mercaptopurine

9
Reserpine
 -> Reserpine

10
Actinomycin D
 -> Actinomycine D

11
Pronetalol hydrochloride
 -> Chlorhydrate de Pronetalol

12
Piperonyl butoxide
 -> Butoxyde de pipéronyle

13
2.4.6-Tris(1-aziridinyl)-s-triazine
 -> 2.4.6-Tris (1-aziridinyl) -s-triazine

14
5-Fluorouracil
 -> 5-fluorouracile

15
Propylthiouracil
 -> Propylthiouracil

16
Nitrogen mustard
 -> Moutarde à l'azote

17
Ethyl carbamate (Urethane)
 -> Carbamate d'éthyle (uréthane)

18
Spironolactone
 -> Spironolactone

19
Thiotepa
 -> Thiotepa

20
Apholate
 -> Apholate

21
Trichlorfon
 -> Trichlorfon



 -> 2-naphtylamine

172
Coumarin
 -> Coumarin

173
3.3'-Dimethoxybenzidine-4.4'-diisocyanate
 -> 3,3'-Diméthoxybenzidine-4,4'-diisocyanate

174
3.3'-Dichlorobenzidine
 -> 3,3'-Dichlorobenzidine

175
4-Aminobiphenyl
 -> 4-aminobiphényle

176
Benzidine
 -> Benzidine

177
4-Nitrobiphenyl
 -> 4-Nitrobiphényle

178
Methyleugenol
 -> Méthyleugénol

179
Benzoyl peroxide
 -> Peroxyde de benzoyle

180
Dihydrosafrole
 -> Dihydrosafrole

181
Safrole
 -> Safrole

182
2.4-D (2.4-dichlorophenoxyacetic acid) (See also Chlorophenoxy herbicides)
 -> 2.4-D (acide 2,4-dichlorophénoxyacétique) (voir aussi Herbicides chlorophénoxy)

183
Sulfallate
 -> Sulfallate

184
ortho-Dichlorobenzene
 -> ortho-dichlorobenzène

185
ortho-Toluidine
 -> ortho-toluidine

186
2.4-Xylidine
 -> 2.4-Xylidine

187
4-Chloro-ortho-toluidine
 -> 4-Chloro-ortho-toluidine

188
2.5-Diaminotoluene
 -> 2,5-diaminotoluène

189
2.5-Xylidine
 -> 2,5-xylidine

190
5-Chloro-ortho-toluidine
 -> 5-Chloro-ortho-toluidine



 -> Ziram

339
Shikimic acid
 -> Acide shikimique

340
Cyclamates (sodium cyclamate)
 -> Cyclamates (cyclamate de sodium)

341
Nitrilotriacetic acid and its salts
 -> Acide nitrilotriacétique et ses sels

342
4.4'-Thiodianiline
 -> 4.4'-Thiodianiline

343
Nithiazide
 -> Nithiazide

344
Benzyl acetate
 -> Acétate de benzyle

345
para-Dimethylaminoazobenzenediazo sodium sulfonate
 -> para-diméthylaminoazobenzénediazo sulfonate de sodium

346
Aramite®
 -> Aramite®

347
Ethyl acrylate
 -> Acrylate d'éthyle

348
n-Butyl acrylate
 -> acrylate de n-butyle

349
3.4-Epoxy-6-methylcyclohexylmethyl-3.4-epoxy-6-methylcyclo-hexanecarboxylate
 -> 3,4-époxy-6-méthylcyclohexylméthyl-3,4-époxy-6-méthylcyclo-hexanecarboxylate

350
Thiouracil
 -> Thiouracil

351
2.4-Hexadienal
 -> 2.4-Hexadienal

352
Chlordecone (Kepone)
 -> Chlordécone (Kepone)

353
Vinblastine sulfate
 -> Sulfate de vinblastine

354
Methyl selenac
 -> Méthyl sélénac

355
Sodium diethyldithiocarbamate
 -> Diéthyldithio

### Second half

In [16]:
translator2 = googletrans.Translator()


print('SECOND HALF')
for index in range(int(data.shape[0] / 2), data.shape[0]):
    print()
    print(index)
    print(data['Agent'].iloc[index])    
    try:
        translated_elem = translator2.translate(data['Agent'].iloc[index], dest='fr', src='en')
        print('\b', '->', translated_elem.text)
        french_data['Agent'].iloc[index] = translated_elem.text
    except:
        time.sleep(10)
        print('Warning: not translated')
        translated_elem = translator2.translate(data['Agent'].iloc[index], dest='fr', src='en')
        print('\b', '->', translated_elem.text)
        french_data['Agent'].iloc[index] = translated_elem.text

SECOND HALF

445
Kojic acid
 -> l'acide kojique

446
Mustard gas
 -> Gaz moutarde

447
Sulfur mustard
 -> Moutarde au soufre

448
Tetranitromethane
 -> Tetranitrométhane

449
Chlorobenzilate
 -> Chlorobenzilate

450
1-Chloro-2-methylpropene
 -> 1-Chloro-2-méthylpropène

451
Citrinin
 -> Citrinin

452
Kaempferol
 -> Kaempferol

453
Merphalan
 -> Merphalan

454
N-[4-(5-Nitro-2-furyl)-2-thiazolyl]acetamide
 -> N- [4- (5-Nitro-2-furyl) -2-thiazolyl] acétamide

455
Chrysoidine
 -> Chrysoidine

456
Ethionamide
 -> Ethionamide

457
1.2-Dimethylhydrazine
 -> 1,2-diméthylhydrazine

458
meta-Dichlorobenzene
 -> méta-dichlorobenzène

459
1.3-Dichloropropene (technical-grade)
 -> 1,3-Dichloropropène (qualité technique)

460
Malonaldehyde
 -> Malonaldéhyde

461
Bis(chloromethyl)ether; chloromethyl methyl ether
 -> Bis (chlorométhyl) éther; chlorométhyl méthyl éther

462
Trichloroacetonitrile
 -> Trichloroacétonitrile

463
Tris(1-aziridinyl)phosphine oxide
 -> Tris (1-aziridinyl) 

 -> Ponceau MX

595
Nafenopin
 -> Nafenopin

596
Isophosphamide
 -> Isophosphamide

597
5-(Morpholinomethyl)-3-[(5-nitrofurfurylidene)­amino]-2-oxazolidinone
 -> 5- (Morpholinométhyl) -3 - [(5-nitrofurfurylidène) amino] -2-oxazolidinone

598
Brilliant Blue FCF. disodium salt
 -> Bleu brillant FCF. sel disodique

599
4.5'.8-Trimethylpsoralen
 -> 4.5'.8-Trimethylpsoralen

600
Crotonaldehyde
 -> Crotonaldéhyde

601
Dacarbazine
 -> Dacarbazine

602
Ponceau SX
 -> Ponceau SX

603
N-Nitrosomethylvinylamine
 -> N-nitrosométhylvinylamine

604
5-Aminoacenaphthene
 -> 5-amino acénaphtène

605
Guinea Green B
 -> Guinée vert B

606
4-Chloro-meta-phenylenediamine
 -> 4-Chloro-méta-phénylènediamine

607
Light Green SF
 -> Vert clair SF

608
D & C Red No. 9
 -> D & C Rouge No. 9

609
1.4-Diamino-2-nitrobenzene
 -> 1,4-diamino-2-nitrobenzène

610
Dibenzo[a.e]fluoranthene
 -> Dibenzo [a.e] fluoranthène

611
Glycidyl oleate
 -> Oléate de glycidyle

612
Ethyl selenac
 -> Ethyl sélénac


 -> Riddelliine

737
Hycanthone mesylate
 -> Hycanthone mésylate

738
Rugulosin
 -> Rugulosine

739
Potassium bis(2-hydroxyethyl)dithiocarbamate
 -> Potassium bis (2-hydroxyéthyl) dithiocarbamate

740
cis-9.10-Epoxystearic acid
 -> acide cis-9.10-époxystéarique

741
para-Aramid fibrils
 -> fibrilles para-aramides

742
Vinyl toluene
 -> Vinyl toluène

743
Butylated hydroxyanisole (BHA)
 -> Hydroxyanisole butylé (BHA)

744
Nylon 6
 -> Nylon 6

745
Acepyrene (3.4-dihydrocyclopenta[cd]pyrene)
 -> Acepyrène (3,4-dihydrocyclopenta [cd] pyrène)

746
Gemfibrozil
 -> Gemfibrozil

747
trans-2-[(Dimethylamino)methylimino]-5-[2-(5-nitro-2-furyl)-vinyl]-1.3.4-oxadiazole
 -> trans-2 - [(Diméthylamino) méthylimino] -5- [2- (5-nitro-2-furyl) -vinyl] -1.3.4-oxadiazole

748
A-alpha-C (2-Amino-9H-pyrido[2.3-b]indole)
 -> A-alpha-C (2-amino-9H-pyrido [2,3-b] indole)

749
Ripazepam
 -> Ripazépam

750
Toluene diisocyanates
 -> Diisocyanates de toluène

751
Hydroxysenkirkine
 -> Hydroxysénkir

 -> Fusarium graminearum. F. culmorum. et F. crookwellense. toxines dérivées de (zéaralénone, deoxynivalénol, nivalénol et fusarénone X)

857
Fusarium sporotrichioides. toxins derived from (T-2 toxin)
 -> Fusarium sporotrichioides. toxines dérivées de (toxine T-2)

858
Goldenseal root powder
 -> Goldenseal poudre de racine

859
Hexachlorocyclohexanes
 -> Hexachlorocyclohexanes

860
Hypochlorite salts
 -> Sels d'hypochlorite

861
Isopropyl oils
 -> Huiles d'isopropyle

862
Madder root (Rubia tinctorum)
 -> Racine de garance (Rubia tinctorum)

863
Mate. not very hot (drinking)
 -> Camarade. pas très chaud (boire)

864
Metabisulfites
 -> Métabisulfites

865
Methylmercury compounds
 -> Composés de méthylmercure

866
Microcystis extracts
 -> Extraits de microcystis

867
Mineral oils. highly-refined
 -> Huiles minérales très raffiné

868
Mineral oils. untreated or mildly treated
 -> Huiles minérales non traité ou légèrement traité

869
Modacrylic fibres
 -> Fibres modacryliques

In [17]:
french_data['Agent']

id
0                                           Formaldéhyde
1                                          Phénobarbital
2                                           Mitomycine C
3                                       Cyclophosphamide
4             DDT (4,4'-dichlorodiphényltrichloroéthane)
5                                       Benzo [a] pyrène
6                                         Phénylbutazone
7                                   Citrate de clomifène
8                                       6-mercaptopurine
9                                              Reserpine
10                                        Actinomycine D
11                            Chlorhydrate de Pronetalol
12                                Butoxyde de pipéronyle
13                 2.4.6-Tris (1-aziridinyl) -s-triazine
14                                       5-fluorouracile
15                                      Propylthiouracil
16                                    Moutarde à l'azote
17                         C

In [19]:
french_data.to_csv('../clean_french_IARC_classification.csv')